In [2]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('plot_ETMC26')
enss=['b','c','d']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
key2bare_A20=yu.load_pkl('pkl/analysis_avgx_2/reg_ignore/key2bare_A20_selected.pkl')
key2bare_B20=yu.load_pkl('pkl/analysis_B20_2/reg_ignore/key2bare_B20_selected.pkl')

# RCs
ens2Njk={ens:len(key2bare_A20[(ens,'j+;conn')]) for ens in enss}

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
path='/p/project1/ngff/li47/code/glwc2/project2/02_discNJN_1D/data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in enss[:1]}
for ens in enss[:1]:
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']+1e-10,ens2Njk[ens])[:,0]

In [4]:
stout=10
key2bare=key2bare_A20
yum.extendBare_avgx(key2bare)
key2phy_A20=yum.convert_key2phy_stout(yum.bareRC2phy_avgx(key2bare,ens2RCs),stout)


key2bare=key2bare_B20
yum.extendBare_avgx(key2bare)
key2phy_B20=yum.convert_key2phy_stout(yum.bareRC2phy_avgx(key2bare,ens2RCs),stout)

key2phy={}
for key in key2phy_B20.keys():
    key2phy[key]=(key2phy_A20[key]+key2phy_B20[key])/2
key2phy_J=key2phy

In [5]:
cases=['avgx','B20','J']
for case in cases:
    key2phy={'avgx':key2phy_A20,'B20':key2phy_B20,'J':key2phy_J}[case]

    list_dic=[{
        'key2phy':key2phy,
        # 'key2phy_pre':key2phy
    }]
    fig,axs=yum.makePlot_a2dependence_avgx(list_dic,case=case,noNumberQ=True)
    # addexp(axs[0,0],axs[1,0])

    yu.finalizePlot(f'a2dep_{case}')

In [6]:
ens2q2gA={
    'b':{'ju':[0.867,0.018],'jd':[-0.408,0.014],'js':[-0.0262,0.0086],'jc':[0.0003,0.0045],'jq':[0.433,0.030],'j-':[1.275,0.023],'j+;conn':[0.585,0.013]},
    'c':{'ju':[0.832,0.022],'jd':[-0.415,0.021],'js':[-0.036,0.017],'jc':[-0.012,0.012],'jq':[0.368,0.065],'j-':[1.247,0.015],'j+;conn':[0.567,0.011]},
    'd':{'ju':[0.843,0.016],'jd':[-0.415,0.015],'js':[-0.034,0.013],'jc':[0.0076,0.0094],'jq':[0.401,0.049],'j-':[1.258,0.014],'j+;conn':[0.5659,0.0072]}
}
ens2rb={ens:{} for ens in enss}
for ens in enss:
    for j in ['jq','ju','jd','js','jc','j+;conn','j-']:
        ens2rb[ens][f'{j}']=yu.jackknife_pseudo([ens2q2gA[ens][j][0]],np.array([[ens2q2gA[ens][j][1]**2+1e-10]]),ens2Njk[ens])[:,0]
    ens2rb[ens][f'ju;conn']=(ens2rb[ens][f'j+;conn']+ens2rb[ens][f'j-'])/2
    ens2rb[ens][f'jd;conn']=(ens2rb[ens][f'j+;conn']-ens2rb[ens][f'j-'])/2
    ens2rb[ens][f'jq;conn']=ens2rb[ens][f'j+;conn']
    
ens2rb['a=#_MA']={}; ens2rb['a=#_const']={}; ens2rb['a=#_linear']={}

a2s_plt=yum.lat_a2s_plt
for j in ['jq','ju','jd','js','jc','ju;conn','jd;conn','jq;conn']:
    a2s=np.array([yu.ens2a[ens]**2 for ens in enss])
    y_jk=yu.superjackknife([ens2rb[ens][f'{j}'][:,None] for ens in enss])
        
    fits=[]
    
    def fitfunc(pars):
        g=pars
        return g+0*a2s
    pars_jk,chi2_jk,Ndof,Nwarning=yu.jackfit(fitfunc,y_jk,[1])
    def fitfunc_plt(pars):
        g=pars
        return g+0*a2s_plt
    pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
    ens2rb['a=#_const'][f'{j}']=pars_plt
    fits.append(['const',pars_plt,chi2_jk,Ndof])
    
    def fitfunc(pars):
        g,c=pars
        return g+c*a2s
    pars_jk,chi2_jk,Ndof,Nwarning=yu.jackfit(fitfunc,y_jk,[1,1])
    def fitfunc_plt(pars):
        g,c=pars
        return g+c*a2s_plt
    pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
    ens2rb['a=#_linear'][f'{j}']=pars_plt
    fits.append(['linear',pars_plt,chi2_jk,Ndof])
    
    pars_plt,props_jk=yu.jackMA(fits)
    ens2rb['a=#_MA'][f'{j}']=pars_plt

# for ft in ['const','linear','MA']:
#     ens2rb[f'a=#_{ft}'][f'jtot;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;{stout}']+ens2rb[f'a=#_{ft}'][f'jg;{stout}']
#     ens2rb[f'a=#_{ft}'][f'jtot;conn;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;conn;{stout}']

#     for fla in fla2iso.keys():
#         ens2rb[f'a=#_{ft}'][f'j{fla};{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
#     for fla in fla2iso_conn.keys():
#         ens2rb[f'a=#_{ft}'][f'j{fla};conn;{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)

key2phy={}
for ens in ens2rb.keys():
    for j in ens2rb[ens].keys():
        key2phy[(ens,j)]=ens2rb[ens][j]
key2phy_gA=key2phy

key2phy_DeltaSigmaBy2={key:val/2 for key,val in key2phy_gA.items()}

key2phy_L={}
for ens in ['a=#_MA','a=#_const','a=#_linear']:
    for j in ['ju','jd','js','jc','jq','ju;conn','jd;conn','jq;conn']:
        key2phy_L[(ens,j)]=key2phy_J[(ens,j)] - key2phy_DeltaSigmaBy2[(ens,j)]

In [7]:
stout=10
cases=['avgx','J','DeltaSigmaBy2','L']
for case in cases:
    key2phy={'avgx':key2phy_A20,'B20':key2phy_B20,'J':key2phy_J,'DeltaSigmaBy2':key2phy_DeltaSigmaBy2,'L':key2phy_L}[case]
    yum.makePlot_percentBar(yum.convert_key2phy_stout(key2phy,stout),case=case)
    yu.finalizePlot(f'bar_{case}')

In [8]:
'''
# from Christos
sgm_piN=39.1(2.5)
sgm_s=36.7(6.7)
sgm_c=86(21)
sgm_sum=164(22)
'''
a=key2phy_A20[('a=#_MA','jq')][:,0]
ag=key2phy_A20[('a=#_MA','jg')][:,0]
Njk=len(a)
Mm=yu.jackknife_pseudo(0.164,0.022,Njk)[:,0] # from Christos above
MN=yu.jackknife_pseudo(0.9362,0.0065,Njk)[:,0] # axial paper: 2309.05774
b=Mm/MN

noNumbersOnBar=False

def makePlot(case):
    def get(j):
        dic={'MN':MN, 'Mm':Mm, 'Mq':3*(a-b)/4*MN, 'Mg':3*ag/4*MN}
        
        if case==1:
            dic['Ma']=(1-b)/4*MN # = (dic['MN']-dic['Mm'])/4
        elif case==2:
            dic['Ma']=dic['MN']-dic['Mm']-dic['Mq']-dic['Mg']
        
        dic['M']=np.sum([dic[j] for j in ['Mm','Ma','Mq','Mg']],axis=0)
        return dic[j]/dic['MN']*100 
                

    fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

    js = ['Mm','Ma','Mq','Mg','M']
    # labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
    labels = [r'$M_m$', r'$M_a$', r'$M_q$', r'$M_g$', r'$M$']
    colors = ['b','orange','green','red','purple']

    ax=axs[0,0]
    ax.axhline(100,color='black',ls='--',marker='')
    # ax.axhline(0,color='black',ls='--',marker='')
    ax.set_ylabel(r'% of M=936(6) MeV')

    x = np.arange(len(labels))
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_xlim([x[0]-3/4,x[-1]+3/4])
    ax.set_ylim([0,105])

    mes=[yu.jackme(get(j)) for j in js]

    factor=100
        
    bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.8, width=0.5, edgecolor='grey')

    func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
    percentages=[func(ele) for ele in mes]

    if not noNumbersOnBar:
        for i, (bar, pct) in enumerate(zip(bars, percentages)):
            height = bar.get_height()
            height = 1 if height < 20 else height - 20
            ax.text(bar.get_x() + 0.6, height, pct,
                    ha='center', va='bottom', fontsize=10, rotation=90)
            
    yu.finalizePlot(f'bar_mass_sumrule{case}')
    
makePlot(1)
makePlot(2)

# Example fits

In [9]:
projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

path=f'pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [10]:
key2bare={}

overwrite=False
symmetrizeQ=True
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    gett2=lambda t:round(t/yu.ens2a[ens]/2)*2
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]

    tfmins_2st=range(8,ens2tfs_conn[ens][-1],2); tcmins_2st=range(2,ens2tfs_conn[ens][-1]//2-1)
    
    pars_jk_meff2st=ens2pars_jk_meff2st[ens]
    fittype='2st2step_SYMshare'
    
    tf2ratio=key2tf2ratio[(ens,j)]
    if symmetrizeQ:
        tf2ratio=yu.symmetrizeRatio(tf2ratio)
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st,tcmins_2st,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    fits_2st=yu.doFits_3pt(fittype,tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=pars_jk_meff2st,symmetrizeQ=symmetrizeQ,label=f'{ens}_{j}_2st_{symmetrizeQ}',overwrite=overwrite)
    # tfmin=gett2(8*yu.ens2a['b']); tcmin=gett(0.2)
    tfmin=gett2(0.7); tcmin=gett(0.2)
    # tfmin=gett2(1.0); tcmin=gett(0.2)
    fit_2st_MA=yu.doMA_3pt(fits_2st,fitlabels=(tfmin,tcmin))

    key2bare[(ens,j)]=fit_2st_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,None,None,fits_sum,fits_2st],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[None,None,None,fit_2st_MA],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,None,gett(0.2),gett(0.2),None,None],
        'fit_2st_rainbow_midpoint:[fittype,pars_jk_meff2st]':[fittype,ens2pars_jk_meff2st[ens]],
        'xunit':yu.ens2a[ens],
    }
    return dic

js_plt=['j+;conn']; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    yu.setpath('analysis_avgx_2')
    list_dic=[createDic(key) for key in keys]
    yu.setpath('plot_ETMC26')

    fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint','fit_2st'],noLegendQ=True,fontsize_colHeaders=32)
    yu.addRowHeader(axs,[yu.ens2label[ens] for ens in enss_plt],fontsize=32)
    yu.finalizePlot(f'rainbow/{j}',mkdirQ=True)

In [11]:
ens2c2pt={}; ens2moms_2pt={}; ens2c2pt0={}; ens2Njk={}
for ens in enss:
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    path=f'{basepath}disc_2pt.h5'
    with h5py.File(path) as f:
        moms_2pt=yu.moms2list(f['moms'])
        c2pt=yu.jackknife(np.real(f['data/N_N'][:,:,:]))
        
    ens2moms_2pt[ens]=moms_2pt
    ens2c2pt[ens]=c2pt
    ens2c2pt0[ens]=c2pt[:,:,moms_2pt.index([0,0,0])]
    ens2Njk[ens]=len(c2pt)

key2tf2ratio={}
stouts=[5,7,10,15,20]
js=['j+;disc','js;disc','jc;disc']+[f'jg;stout{stout}' for stout in stouts]
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{yu.ens2full[ens]}/data_merge/disc_0,0,1,0,0,0.h5'
    with h5py.File(path) as f:
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            if j not in js:
                continue        
            key=(ens,j)
            if key not in key2tf2ratio:
                key2tf2ratio[key]={}
            c3pt=yu.jackknife(f['data'][jtf][:,:,0,projs.index('P0'),inserts.index('tz')])
            c2pt=ens2c2pt[ens][:,tf,ens2moms_2pt[ens].index([0,0,1])]
            factor=1/(1j*2*np.pi/yu.ens2NL[ens])
            ratio=np.real(c3pt/c2pt[:,None]*factor)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_disc={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;disc')].keys()); tfs.sort()
    ens2tfs_disc[ens]=tfs
    print(ens,tfs[0],tfs[-1])
    
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

b 2 22
c 2 26
d 2 30


In [12]:
selections=['bandfit_WA','const_MA','sum_MA','2st_MA']
selection2key2bare={selection:{} for selection in selections}

overwrite=False
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.3,1.25)
    tcmins_1st=get_tfs(2*lat_a,0.6)
    tfmins_sum=get_tfs(0.2,1.0); tcmins_sum=[2]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.3))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band')
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.7),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,corrQ=True,label=f'{ens}_{j}_const',overwrite=overwrite)
    tfmin=gett(0.6); tcmin=gett(0.3)
    fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_sum,tcmins_sum,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    tfmin=gett(0.35); tcmin=2
    fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max,dtf,dtc]':[gett(0.5),gett(1.3),gett(0.2),gett(0.2),None,None],
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,None,None,None,None,None],
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,None,None,None,None,None],
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,None,None,None,None,None],
        'xunit':yu.ens2a[ens],
    }
    return dic

def createDic2(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.3,1.25)
    tcmins_1st=get_tfs(2*lat_a,0.6)
    tfmins_sum=get_tfs(0.2,1.0); tcmins_sum=[2]
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.3))
    fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_band')
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.7),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,corrQ=True,label=f'{ens}_{j}_const',overwrite=overwrite)
    tfmin=gett(0.6); tcmin=gett(0.3)
    fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
    fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_sum,tcmins_sum,corrQ=False,label=f'{ens}_{j}_sum',overwrite=overwrite)
    tfmin=gett(0.35); tcmin=2
    fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
        'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[None,fit_const_MA,None,None],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max,dtf,dtc]':[gett(0.5),gett(1.3),gett(0.2),gett(0.2),None,None],
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,None,gett(0.3),gett(0.3),None,None],
        'xunit':yu.ens2a[ens],
    }
    return dic

# js_plt=['j+;disc']; enss_plt=['b','c','d']
# js_plt=['jg;stout10']; enss_plt=['b','c','d']
js_plt=['jg;stout10']; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    yu.setpath('analysis_avgx_2')
    list_dic=[createDic2(key) if key[1].startswith('jg') else createDic(key) for key in keys]
    yu.setpath('plot_ETMC26')
    
    fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow', 'fit_band', 'fit_const'],fontsize_colHeaders=32)
    yu.addRowHeader(axs,[yu.ens2label[ens] for ens in enss_plt],fontsize=32)
    yu.finalizePlot(f'rainbow/{j}',mkdirQ=True)

# stout dependence

In [18]:
yu.setpath('analysis_avgx_2')
tftc2key2bare=yu.load_pkl_reg('tftc2key2bare_allstouts')
ens2Njk={ens:tftc2key2bare[list(tftc2key2bare.keys())[0]][(ens,'j+;conn')].shape[0] for ens in enss}

# RCs
path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
path='data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in ens2RCs_pre_me.keys()}
for ens in ens2RCs_pre_me.keys():
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
tftcs=list(tftc2key2bare.keys()); tftcs.sort()

def run(tftc):
    key2bare=tftc2key2bare[tftc]
    yum.extendBare_avgx(key2bare)
    key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)
    key2phy_pre=yum.bareRC2phy_avgx_pre(key2bare,ens2RCs,ens2RCs_pre)
    return key2phy,key2phy_pre

tasks=tftcs
results=yu.parallelizeTasks(run,tasks,max_workers=13)

tftc2key2phy={}; tftc2key2phy_pre={}
for task,res in zip(tasks,results):
    tftc2key2phy[task],tftc2key2phy_pre[task]=res
    
    
PDF_SET='NNPDF40_nnlo_as_01180'
PDF_SET='JAM22-PDF_proton_nlo'

j2me=yu.load_pkl(f'pkl/lhapdf/reg_ignore/j2me_{PDF_SET}.pkl')
j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

def addexp(ax0,ax1):
    ax=ax0
    for j in ['jtot','jq','jg']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)
    ax=ax1
    for j in ['ju','jd','js','jc']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)

def addexp_span(ax0,ax1):
    ax=ax0
    for j in ['jtot','jq','jg']:
        m,e=j2me[j]
        ax.axhspan(m-e,m+e,color=j2color[j],alpha=0.4)
    ax=ax1
    for j in ['ju','jd','js','jc']:
        m,e=j2me[j]
        ax.axhspan(m-e,m+e,color=j2color[j],alpha=0.4)
        
tftc_selected=(0.6,0.3)

yu.setpath('plot_ETMC26')

26/26 tasks completed          


In [19]:
tftc2key2phy[tftc_selected].keys()

dict_keys([('c', 'jv1;conn'), ('c', 'jv1;disc'), ('c', 'jv1'), ('c', 'jv2;conn'), ('c', 'jv2;disc'), ('c', 'jv2'), ('c', 'jv3;conn'), ('c', 'jv3;disc'), ('c', 'jv3'), ('c', 'jq;conn'), ('c', 'jq;conn;stout0'), ('c', 'jq;disc;stout0'), ('c', 'jq;0mix;stout0'), ('c', 'jq;mix;stout0'), ('c', 'jq;stout0'), ('c', 'jg;0mix;stout0'), ('c', 'jg;mix;stout0'), ('c', 'jg;stout0'), ('c', 'jtot;stout0'), ('c', 'ju;stout0'), ('c', 'jd;stout0'), ('c', 'js;stout0'), ('c', 'jc;stout0'), ('c', 'jq;conn;stout1'), ('c', 'jq;disc;stout1'), ('c', 'jq;0mix;stout1'), ('c', 'jq;mix;stout1'), ('c', 'jq;stout1'), ('c', 'jg;0mix;stout1'), ('c', 'jg;mix;stout1'), ('c', 'jg;stout1'), ('c', 'jtot;stout1'), ('c', 'ju;stout1'), ('c', 'jd;stout1'), ('c', 'js;stout1'), ('c', 'jc;stout1'), ('c', 'jq;conn;stout2'), ('c', 'jq;disc;stout2'), ('c', 'jq;0mix;stout2'), ('c', 'jq;mix;stout2'), ('c', 'jq;stout2'), ('c', 'jg;0mix;stout2'), ('c', 'jg;mix;stout2'), ('c', 'jg;stout2'), ('c', 'jtot;stout2'), ('c', 'ju;stout2'), ('c',

In [22]:
stouts_plt=[5,10,20]
figs=[]
for ce in ['MA']:
    tftc=tftc_selected
    list_dic=[
        {'key2phy':yum.convert_key2phy_stout(tftc2key2phy[tftc],stout),
        # 'key2phy_pre':yum.convert_key2phy_stout(tftc2key2phy_pre[tftc],stout)
        }
        for stout in stouts_plt
    ]
    fig,axs=yum.makePlot_a2dependence_avgx(list_dic,ce=ce, noNumberQ=True)
    
    # for i in range(len(list_dic)):
    #     addexp(axs[0,i],axs[1,i])
        
    # fig.suptitle(f'{ce}')
    yu.addColHeader(axs,[f'stout{stout}' for stout in stouts_plt],fontsize=24)
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
    
yu.makePDF('a2dep_stout',figs)

In [48]:
stouts_plt=range(2,41,2)
xs=stouts_plt

fig,axs=yu.getFigAxs(1,1)
ax=axs[0,0]

ind=xs.index(10)

ys=np.concatenate([tftc2key2phy[tftc_selected]['a=#_MA',f'jtot;stout{stout}'][:,:1] for stout in stouts_plt],axis=1)
mean,err=yu.jackme(ys)
ax.errorbar(xs,mean,err,color='grey',label=r'$\langle x \rangle_N$')
ax.errorbar(xs[ind],mean[ind],err[ind],color='grey',mfc='white')


ys=np.concatenate([tftc2key2phy[tftc_selected]['a=#_MA',f'jq;stout{stout}'][:,:1] for stout in stouts_plt],axis=1)
mean,err=yu.jackme(ys)
ax.errorbar(xs,mean,err,color='purple',label=r'$\langle x \rangle_q$')
ax.errorbar(xs[ind],mean[ind],err[ind],color='purple',mfc='white')

ys=np.concatenate([tftc2key2phy[tftc_selected]['a=#_MA',f'jg;stout{stout}'][:,:1] for stout in stouts_plt],axis=1)
mean,err=yu.jackme(ys)
ax.errorbar(xs,mean,err,color='cyan',label=r'$\langle x \rangle_g$')
ax.errorbar(xs[ind],mean[ind],err[ind],color='cyan',mfc='white')

yu.addRefLine(ax,1)
ax.legend(ncols=3)
ax.set_xlabel(r'$N_{stout}$')
ax.set_ylabel(r'$\langle x \rangle_{q,g}$')
yu.finalizePlot('stoutDep')

In [69]:
stout=10
case='avgx'
key2phy={'avgx':key2phy_A20,'B20':key2phy_B20,'J':key2phy_J,'DeltaSigmaBy2':key2phy_DeltaSigmaBy2,'L':key2phy_L}[case]
fig,axs=yum.makePlot_percentBar(yum.convert_key2phy_stout(key2phy,stout),case=case)
ax=axs[0,0]
ax.set_ylim([0,1.2])

j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

PDF_SET='JAM22-PDF_proton_nlo'
j2me=yu.load_pkl(f'pkl/lhapdf/reg_ignore/j2me_{PDF_SET}.pkl')
js=['ju','jd','js','jc','jq','jg','jtot']
for ij,j in enumerate(js):
    m,e=j2me[j]
    ax.errorbar(ij+0.3,m,e,color=j2color[j],markersize=5,label=None if j!='jtot' else 'JAM22')
    
PDF_SET='NNPDF40_nnlo_as_01180'
j2me=yu.load_pkl(f'pkl/lhapdf/reg_ignore/j2me_{PDF_SET}.pkl')
js=['ju','jd','js','jc','jq','jg','jtot']
for ij,j in enumerate(js):
    m,e=j2me[j]
    ax.errorbar(ij+0.5,m,e,color=j2color[j],markersize=5,mfc='white',label=None if j!='jtot' else 'NNPDF4.1')

ax.legend()
# fig.suptitle('stout10')
yu.finalizePlot(f'bar_{case}_withEXP')